In [1]:
# Sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import shap
import pickle

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import tqdm

In [3]:
import pandas as pd
import numpy as np

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
class Linear(nn.Module):
    def __init__(self, weights,bias):
        super(Linear, self).__init__()
        self.weights = weights
        self.bias = bias        
    
    def forward(self, x):
        y = torch.matmul(x,self.weights)+self.bias
        return y

In [6]:
def min_max_normalization(df,col_name):
    min_val = df[col_name].min()
    max_val = df[col_name].max()
    df[col_name] = (df[col_name]-min_val)/(max_val-min_val)
    return df

In [7]:
def conterfactual_infer(query_x,target_y,feature_ids, model, loss_fn, max_iters):
    model.eval()
    tqdm_range = tqdm.tqdm(range(max_iters))
    cx = torch.clone(query_x)
    cx.requires_grad = True
    cf_optimizer = torch.optim.Adam([cx], lr=0.01)
    mask = torch.ones_like(query_x).type(torch.bool)
    mask[:,feature_ids] = False

    for it in tqdm_range:
        cy = model(cx)
        loss = loss_fn(cy,target_y)#loss_fn(query_x,cx,cy,target_y,feature_ids)
        loss.backward()
        cx.grad[mask]=0.
        cf_optimizer.step()
        cf_optimizer.zero_grad()

        if (it+1)%10 == 0:    
            tqdm_range.write('iter: {}, loss: {}'.format(it+1, loss.detach()))
    return cx, model(cx)

In [36]:
def query_recommendation(query_x,target_y,model,query_features,max_iters=200):
    
    if len(query_x.shape)==1:
        query_x = query_x.reshape(1,-1)
    if not isinstance(query_x,torch.Tensor):
        query_x = torch.Tensor(query_x)
        
    query_y = model(query_x)
    feature_ids = [feature_map[s] for s in query_features ]
    print(feature_ids)
    loss_fn = torch.nn.MSELoss(reduction='mean')
    
    cx,cy = conterfactual_infer(query_x,target_y,feature_ids, model, loss_fn, max_iters)
    
    diff = torch.abs(query_x-cx).reshape(-1)
    

    cf = features[diff>0]
    cx = cx[:,diff>0]
    
    return cx.detach().numpy().reshape(-1),cy.detach().numpy(),cf

In [9]:
dataset = pd.read_csv("../data/trial_static_data.csv")
dataset.head()

,n_participant,duration,age>65,male,ethnicity_white,use_technology,medication,follow_up_considered,use_wearables,use_mobile_app,...,feedback_provided,prioir_treatment_history_considered,co_mobidities_considered,treatment_type,biosamples_collected,randomised,mutli_site,adverse_event_considered,support_sessions,retention_rate
0,11550.0,418.0,0.269070,0.349416,0.521052,yes,Testosterone,yes,no,no,...,yes,no,yes,behavioural,no,yes,no,yes,yes,0.545453
1,60926.0,56.0,0.355991,0.461932,0.437640,no,Testosterone,yes,yes,yes,...,no,yes,yes,behavioural,yes,yes,yes,yes,yes,0.911800
2,13382.0,373.0,0.404902,0.304780,0.819994,yes,5ARI,yes,yes,yes,...,yes,no,yes,behavioural,no,yes,no,no,yes,0.597245
3,24097.0,183.0,0.264093,0.620426,0.824459,yes,venlafaxine,yes,no,no,...,yes,yes,yes,medication,yes,no,yes,yes,yes,0.780205
4,32748.0,182.0,0.114587,0.412570,0.702339,no,citalopram,yes,yes,yes,...,no,yes,no,rTMS,no,no,no,no,yes,0.773729


In [10]:
continuous = ["n_participant","duration","age>65","male","ethnicity_white","retention_rate"]
categorical = dataset.columns.difference(continuous)

In [11]:
one_hot_encoded = pd.get_dummies(dataset[categorical])
df_encoded = pd.concat([dataset.drop(categorical,axis=1), one_hot_encoded], axis=1)
df_encoded = min_max_normalization(df_encoded,["n_participant","duration"])

In [12]:
df_encoded.head()

,n_participant,duration,age>65,male,ethnicity_white,retention_rate,adverse_event_considered_no,adverse_event_considered_yes,biosamples_collected_no,biosamples_collected_yes,...,treatment_type_behavioural,treatment_type_medication,treatment_type_observational,treatment_type_rTMS,use_mobile_app_no,use_mobile_app_yes,use_technology_no,use_technology_yes,use_wearables_no,use_wearables_yes
0,0.115041,0.834677,0.269070,0.349416,0.521052,0.545453,0,1,1,0,...,1,0,0,0,1,0,0,1,1,0
1,0.609063,0.104839,0.355991,0.461932,0.437640,0.911800,0,1,0,1,...,1,0,0,0,0,1,1,0,0,1
2,0.133371,0.743952,0.404902,0.304780,0.819994,0.597245,1,0,1,0,...,1,0,0,0,0,1,0,1,0,1
3,0.240578,0.360887,0.264093,0.620426,0.824459,0.780205,0,1,0,1,...,0,1,0,0,1,0,0,1,1,0
4,0.327133,0.358871,0.114587,0.412570,0.702339,0.773729,1,0,1,0,...,0,0,0,1,0,1,1,0,0,1


In [13]:
target = dataset["retention_rate"]
train_dataset, test_dataset, y_train, y_test = train_test_split(df_encoded,
                                                                target,
                                                                test_size=0.2,
                                                                random_state=0,
                                                                )
x_train = train_dataset.drop('retention_rate', axis=1)
x_test = test_dataset.drop('retention_rate', axis=1)


In [14]:
features = x_train.columns
feature_map = {}
for i,f in enumerate(features):
    feature_map[f]=i

In [15]:
features

Index(['n_participant', 'duration', 'age>65', 'male', 'ethnicity_white',
       'adverse_event_considered_no', 'adverse_event_considered_yes',
       'biosamples_collected_no', 'biosamples_collected_yes',
       'co_mobidities_considered_no', 'co_mobidities_considered_yes',
       'feedback_provided_no', 'feedback_provided_yes',
       'follow_up_considered_no', 'follow_up_considered_yes',
       'interviews_needed_no', 'interviews_needed_yes', 'medication_5ARI',
       'medication_Antidepressant', 'medication_Escitalopram',
       'medication_Testosterone', 'medication_citalopram',
       'medication_venlafaxine', 'mutli_site_no', 'mutli_site_yes',
       'prioir_treatment_history_considered_no',
       'prioir_treatment_history_considered_yes', 'questionare_completed_no',
       'questionare_completed_yes', 'randomised_no', 'randomised_yes',
       'support_sessions_no', 'support_sessions_yes',
       'treatment_type_behavioural', 'treatment_type_medication',
       'treatment_type_o

In [16]:
lrg = LinearRegression()
lrg.fit(x_train,y_train)

LinearRegression()

In [17]:
pred_y = lrg.predict(x_test)
score = mean_absolute_error(y_test,pred_y)
score

2.527256182105475e-16

In [18]:
lrg.coef_,lrg.intercept_

(array([ 3.88139867e-16, -4.96000000e-01,  5.00000000e-02, -8.56018295e-17,
         1.98603175e-16, -3.03069803e-05, -3.03069803e-05, -7.94522486e-03,
        -7.94522486e-03,  5.00151578e-04,  5.00151578e-04,  7.59676592e-04,
         7.59676592e-04, -2.24458253e-04, -2.24458253e-04,  4.38682655e-04,
         4.38682655e-04,  3.47653041e-05,  3.47653041e-05,  3.47653041e-05,
         3.47653041e-05,  3.47653041e-05,  3.47653041e-05,  1.57152940e-04,
         1.57152940e-04, -2.39808261e-05, -2.39808261e-05, -1.08169612e-03,
        -1.08169612e-03,  1.61616325e-03,  1.61616325e-03, -1.99273153e-02,
         3.00726847e-02,  3.46264591e-04,  3.46264591e-04,  3.46264591e-04,
         3.46264591e-04, -3.40628052e-04, -3.40628052e-04,  2.28732710e-03,
         2.28732710e-03, -6.24437967e-04, -6.24437967e-04]),
 0.9200578643360325)

In [19]:
explainer = shap.Explainer(lrg.predict, x_test)
# Calculates the SHAP values - It takes some time
shap_values = explainer(x_test)
shap.summary_plot(shap_values, plot_type='violin',feature_names = features)

Permutation explainer: 10001it [01:50, 82.44it/s]                                                                                                                                  
matplotlib is not installed so plotting is not available! Run `pip install matplotlib` to fix this.


In [20]:
model = Linear(weights=torch.Tensor(lrg.coef_),bias=torch.ones(1)*lrg.intercept_)

In [21]:
pickle.dump(lrg, open('model-lr', 'wb'))

In [22]:
### an example of query recommendation

x_test_ts = torch.Tensor(x_test.values.astype(np.float32))
y_test_ts = torch.Tensor(y_test.values.astype(np.float32))
query_x = x_test_ts[1].reshape(1,-1)
qy = model(query_x)

torch.save(model.state_dict(), 'model_weights.pth')

In [23]:
query_features = ["n_participant","duration","age>65","male","ethnicity_white"]
target_y = 0.9
query_x.shape[0]

1

In [37]:
cx, cy, cf = query_recommendation(query_x,target_y=torch.ones(query_x.shape[0])*target_y,query_features=query_features,model=model)

[0, 1, 2, 3, 4]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 4032.17it/s]

iter: 10, loss: 0.03728828951716423
iter: 20, loss: 0.0200056042522192
iter: 30, loss: 0.00909280963242054
iter: 40, loss: 0.0033033997751772404
iter: 50, loss: 0.0008612850215286016
iter: 60, loss: 0.00012079372390871868
iter: 70, loss: 9.189552088173514e-07
iter: 80, loss: 9.478388165007345e-06
iter: 90, loss: 1.2801709090126678e-05
iter: 100, loss: 6.441572622861713e-06
iter: 110, loss: 1.5499309711231035e-06
iter: 120, loss: 9.324142524747003e-08
iter: 130, loss: 2.416436473140493e-08
iter: 140, loss: 6.691749376841472e-08
iter: 150, loss: 3.4738892651375863e-08
iter: 160, loss: 6.209020853020775e-09
iter: 170, loss: 2.0520474208751693e-11
iter: 180, loss: 5.856151119587594e-10
iter: 190, loss: 5.238689482212067e-10
iter: 200, loss: 1.2423484463397472e-10


In [48]:
qstr = " ".join([cf[i]+" --> "+str(cx[i])+";" for i in range(len(cf))])
print("The recommendations is to change:"+ qstr + '\n The new retention rate would be '+str(cy[0].round(3)))

The recommendations is to change:duration --> 0.15574692; age>65 --> 0.62520736;
 The new retention rate would be 0.9
